In [55]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\visha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [56]:
df = pd.read_csv("IMDB.csv")
df.shape

(1000, 2)

In [57]:
df = df.sample(500)
df.shape

(500, 2)

In [58]:
df.to_csv("data.csv", index=False)
df.head()

,review,sentiment
808,"Hmmm, yeah this episode is extremely underrate...",positive
682,Um .... a serious film about troubled teens in...,positive
589,"I may be biased, I am the author of the novel ...",positive
936,This movie is all about entertainment. Imagine...,positive
472,Netflix should mention this short feature on t...,positive


## Data preprocessing

In [59]:
# Define text preprocessing functions

def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r"http[s]?://\S+|www\.\S+")
    return re.sub(url_pattern, "", text)
    
def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise



In [60]:
df = normalize_text(df)
df.head()

,review,sentiment
808,hmmm yeah episode extremely underratedbr br ev...,positive
682,um serious film troubled teen singapore countr...,positive
589,may biased author novel hungry bachelor club s...,positive
936,movie entertainment imagine friend love spendi...,positive
472,netflix mention short feature info silk stocki...,positive


In [61]:
df['sentiment'].value_counts()

sentiment
negative    260
positive    240
Name: count, dtype: int64

In [62]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [63]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
808,hmmm yeah episode extremely underratedbr br ev...,1
682,um serious film troubled teen singapore countr...,1
589,may biased author novel hungry bachelor club s...,1
936,movie entertainment imagine friend love spendi...,1
472,netflix mention short feature info silk stocki...,1


In [64]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

## Feature Engineering

In [65]:
vectorizer = CountVectorizer(max_features=100)

X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [67]:
import dagshub
dagshub.init(repo_owner='vishalchauhan91196', repo_name='MLOps-Production-System', mlflow=True)

mlflow.set_tracking_uri("https://dagshub.com/vishalchauhan91196/MLOps-Production-System.mlflow")
mlflow.set_experiment("Logistic Regression Baseline")

2025-12-16 20:17:40,857 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/vishalchauhan91196/MLOps-Production-System "HTTP/1.1 200 OK"


Initialized MLflow to track repo "vishalchauhan91196/MLOps-Production-System"

2025-12-16 20:17:40,892 - INFO - Initialized MLflow to track repo "vishalchauhan91196/MLOps-Production-System"


Repository vishalchauhan91196/MLOps-Production-System initialized!

2025-12-16 20:17:40,895 - INFO - Repository vishalchauhan91196/MLOps-Production-System initialized!


<Experiment: artifact_location='mlflow-artifacts:/411e33a2042041a99bbf434ce54f4725', creation_time=1765876706045, experiment_id='0', last_update_time=1765876706045, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [69]:
import logging
import os
import time

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()

    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_params({"vectorizer": "Bag of Words",
                           "max_features": 100,
                           "test_size": 0.25})

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)
        
        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metrics({"accuracy": accuracy,
                            "precision": precision,
                            "recall": recall,
                            "f1_score": f1})

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")


    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)    

2025-12-16 20:25:11,725 - INFO - Starting MLflow run...
2025-12-16 20:25:13,454 - INFO - Logging preprocessing parameters...
2025-12-16 20:25:13,922 - INFO - Initializing Logistic Regression model...
2025-12-16 20:25:13,923 - INFO - Fitting the model...
2025-12-16 20:25:13,971 - INFO - Model training complete.
2025-12-16 20:25:13,972 - INFO - Logging model parameters...
2025-12-16 20:25:14,379 - INFO - Making predictions...
2025-12-16 20:25:14,381 - INFO - Calculating evaluation metrics...
2025-12-16 20:25:14,390 - INFO - Logging evaluation metrics...
2025-12-16 20:25:14,948 - INFO - Saving and logging the model...
2025/12/16 20:25:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-16 20:25:29,104 - INFO - Model training and logging completed in 15.65 seconds.
2025-12-16 20:25:29,105 - INFO - Accuracy: 0.66
2025-12-16 20:25:29,106 - INFO - Precision: 0.625
2025-12-16 20:25:29,106 - INFO - Recall: 0.6521739130434783
2025-12-16 20:25:29,107 

🏃 View run marvelous-dove-371 at: https://dagshub.com/vishalchauhan91196/MLOps-Production-System.mlflow/#/experiments/0/runs/9836165310ea456999b027febb20ba65
🧪 View experiment at: https://dagshub.com/vishalchauhan91196/MLOps-Production-System.mlflow/#/experiments/0
